In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../data"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

from sklearn.metrics import mean_squared_error
from sklearn import model_selection
from catboost import CatBoostRegressor
from sklearn.utils import shuffle
from sklearn.model_selection import RandomizedSearchCV



train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")
train = pd.concat([train.drop(['spacegroup'], axis=1), 
                   pd.get_dummies(train['spacegroup'], prefix='SG')], axis=1)
test = pd.concat([test.drop(['spacegroup'], axis=1), 
                   pd.get_dummies(test['spacegroup'], prefix='SG')], axis=1)
train = pd.concat([train.drop(['number_of_total_atoms'], axis=1), 
                   pd.get_dummies(train['number_of_total_atoms'], prefix='NAT')], axis=1)
test = pd.concat([test.drop(['number_of_total_atoms'], axis=1), 
                   pd.get_dummies(test['number_of_total_atoms'], prefix='NAT')], axis=1)


train = shuffle(train, random_state=777)
test_id=test.id


X = train.drop(['id','bandgap_energy_ev','formation_energy_ev_natom'], axis=1)
Y_feen = np.log(train['formation_energy_ev_natom']+1)
Y_bee = np.log(train['bandgap_energy_ev']+1)

test = test.drop(['id'], axis = 1)




sample_submission.csv.zip
test
test.csv
test.csv.zip
train
train.csv
train.csv.zip



In [ ]:



def runCatBoost(x_train, y_train,x_test, y_test,test,depth):
    model=CatBoostRegressor(iterations=3000,
                            learning_rate=0.005,
                            depth=depth,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=99,
                            od_type='Iter',
                            od_wait=50)
    model.fit(x_train, y_train, eval_set=(x_test, y_test), use_best_model=True, verbose=False)
    y_pred_train=model.predict(x_test)
    mse=mean_squared_error(y_test, y_pred_train)
    y_pred_test=model.predict(test)
    return y_pred_train,mse,y_pred_test

kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=30)
for dev_index, val_index in kf.split(X):
    dev_X, val_X = X.loc[dev_index], X.loc[val_index]
    dev_y, val_y = Y_feen.loc[dev_index], Y_feen.loc[val_index]
    y_pred_feen,mse_feen,y_pred_test_feen=runCatBoost(dev_X, dev_y, val_X, val_y,test,depth=15)
print("Mean cv score : ", np.mean(mse_feen))





/usr/local/lib/python3.5/dist-packages/catboost/core.py:533: FutureWarning: The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').
  warnings.warn("The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').", FutureWarning)


0: learn: 0.188314	test: 0.1882659	bestTest: 0.1882659 (0)	total: 48.8ms	remaining: 2m 26s
1: learn: 0.1874937	test: 0.1874555	bestTest: 0.1874555 (1)	total: 75.1ms	remaining: 1m 52s
2: learn: 0.1866739	test: 0.1866473	bestTest: 0.1866473 (2)	total: 120ms	remaining: 1m 59s
3: learn: 0.1858463	test: 0.185835	bestTest: 0.185835 (3)	total: 138ms	remaining: 1m 43s
4: learn: 0.1850354	test: 0.1850275	bestTest: 0.1850275 (4)	total: 188ms	remaining: 1m 52s
5: learn: 0.1842094	test: 0.1842096	bestTest: 0.1842096 (5)	total: 293ms	remaining: 2m 26s
6: learn: 0.1834518	test: 0.183449	bestTest: 0.183449 (6)	total: 308ms	remaining: 2m 11s
7: learn: 0.1826479	test: 0.1826416	bestTest: 0.1826416 (7)	total: 337ms	remaining: 2m 5s
8: learn: 0.1818563	test: 0.1818453	bestTest: 0.1818453 (8)	total: 356ms	remaining: 1m 58s
9: learn: 0.1810786	test: 0.1810654	bestTest: 0.1810654 (9)	total: 375ms	remaining: 1m 52s
10: learn: 0.1802772	test: 0.1802569	bestTest: 0.1802569 (10)	total: 561ms	remaining: 2m 32s
1

92: learn: 0.1275393	test: 0.1278182	bestTest: 0.1278182 (92)	total: 6.92s	remaining: 3m 36s
93: learn: 0.1270429	test: 0.1273236	bestTest: 0.1273236 (93)	total: 7.01s	remaining: 3m 36s
94: learn: 0.1265115	test: 0.1268153	bestTest: 0.1268153 (94)	total: 8.31s	remaining: 4m 14s
95: learn: 0.1260141	test: 0.1263232	bestTest: 0.1263232 (95)	total: 8.33s	remaining: 4m 11s
96: learn: 0.125477	test: 0.1257975	bestTest: 0.1257975 (96)	total: 8.35s	remaining: 4m 9s
97: learn: 0.1249488	test: 0.1252766	bestTest: 0.1252766 (97)	total: 8.52s	remaining: 4m 12s
98: learn: 0.1244426	test: 0.1247779	bestTest: 0.1247779 (98)	total: 8.56s	remaining: 4m 10s
99: learn: 0.1239672	test: 0.1243063	bestTest: 0.1243063 (99)	total: 8.57s	remaining: 4m 8s
100: learn: 0.1234436	test: 0.1237902	bestTest: 0.1237902 (100)	total: 8.61s	remaining: 4m 7s
101: learn: 0.1229341	test: 0.1232931	bestTest: 0.1232931 (101)	total: 8.63s	remaining: 4m 5s
102: learn: 0.1224556	test: 0.1228233	bestTest: 0.1228233 (102)	total: 

180: learn: 0.09040604	test: 0.0913146	bestTest: 0.0913146 (180)	total: 15.5s	remaining: 4m 1s
181: learn: 0.09008271	test: 0.09099189	bestTest: 0.09099189 (181)	total: 15.5s	remaining: 4m
182: learn: 0.08974797	test: 0.09066099	bestTest: 0.09066099 (182)	total: 15.5s	remaining: 3m 59s
183: learn: 0.08941188	test: 0.09033069	bestTest: 0.09033069 (183)	total: 15.6s	remaining: 3m 58s
184: learn: 0.08910886	test: 0.09003178	bestTest: 0.09003178 (184)	total: 15.6s	remaining: 3m 57s
185: learn: 0.08877368	test: 0.08970486	bestTest: 0.08970486 (185)	total: 15.6s	remaining: 3m 56s
186: learn: 0.0884576	test: 0.08938619	bestTest: 0.08938619 (186)	total: 15.6s	remaining: 3m 55s
187: learn: 0.08812792	test: 0.0890711	bestTest: 0.0890711 (187)	total: 15.6s	remaining: 3m 54s
188: learn: 0.08779661	test: 0.0887456	bestTest: 0.0887456 (188)	total: 15.7s	remaining: 3m 53s
189: learn: 0.08745676	test: 0.08842058	bestTest: 0.08842058 (189)	total: 15.8s	remaining: 3m 52s
190: learn: 0.08712547	test: 0.0

271: learn: 0.06609349	test: 0.06787931	bestTest: 0.06787931 (271)	total: 26.2s	remaining: 4m 22s
272: learn: 0.06587554	test: 0.06767584	bestTest: 0.06767584 (272)	total: 26.2s	remaining: 4m 21s
273: learn: 0.06566699	test: 0.06747743	bestTest: 0.06747743 (273)	total: 26.2s	remaining: 4m 20s
274: learn: 0.06544678	test: 0.06727637	bestTest: 0.06727637 (274)	total: 26.3s	remaining: 4m 20s
275: learn: 0.06524384	test: 0.06708361	bestTest: 0.06708361 (275)	total: 26.6s	remaining: 4m 22s
276: learn: 0.0650541	test: 0.06689695	bestTest: 0.06689695 (276)	total: 26.6s	remaining: 4m 21s
277: learn: 0.06487261	test: 0.06671911	bestTest: 0.06671911 (277)	total: 26.7s	remaining: 4m 20s
278: learn: 0.0646637	test: 0.06651965	bestTest: 0.06651965 (278)	total: 26.8s	remaining: 4m 21s
279: learn: 0.06445031	test: 0.06632545	bestTest: 0.06632545 (279)	total: 26.9s	remaining: 4m 21s
280: learn: 0.06425148	test: 0.06613764	bestTest: 0.06613764 (280)	total: 26.9s	remaining: 4m 20s
281: learn: 0.06409615

...

1014: learn: 0.03095968	test: 0.03158017	bestTest: 0.03158017 (1014)	total: 2m 44s	remaining: 5m 21s
1015: learn: 0.03095317	test: 0.03157299	bestTest: 0.03157299 (1015)	total: 2m 44s	remaining: 5m 20s
1016: learn: 0.03094614	test: 0.03156826	bestTest: 0.03156826 (1016)	total: 2m 44s	remaining: 5m 20s
1017: learn: 0.03094173	test: 0.03156889	bestTest: 0.03156826 (1016)	total: 2m 44s	remaining: 5m 20s
1018: learn: 0.03094112	test: 0.03156865	bestTest: 0.03156826 (1016)	total: 2m 44s	remaining: 5m 19s
1019: learn: 0.03093337	test: 0.03156353	bestTest: 0.03156353 (1019)	total: 2m 44s	remaining: 5m 19s
1020: learn: 0.03092927	test: 0.03156213	bestTest: 0.03156213 (1020)	total: 2m 44s	remaining: 5m 18s
1021: learn: 0.0309281	test: 0.03156172	bestTest: 0.03156172 (1021)	total: 2m 44s	remaining: 5m 18s
1022: learn: 0.03092549	test: 0.03155902	bestTest: 0.03155902 (1022)	total: 2m 44s	remaining: 5m 18s
1023: learn: 0.03091018	test: 0.0315522	bestTest: 0.0315522 (1023)	total: 2m 44s	remaining: 

1096: learn: 0.03058961	test: 0.03137872	bestTest: 0.03137872 (1096)	total: 3m 5s	remaining: 5m 22s
1097: learn: 0.03058664	test: 0.03137565	bestTest: 0.03137565 (1097)	total: 3m 5s	remaining: 5m 22s
1098: learn: 0.03058063	test: 0.0313723	bestTest: 0.0313723 (1098)	total: 3m 6s	remaining: 5m 21s
1099: learn: 0.030579	test: 0.03137214	bestTest: 0.03137214 (1099)	total: 3m 6s	remaining: 5m 21s
1100: learn: 0.030575	test: 0.03136773	bestTest: 0.03136773 (1100)	total: 3m 6s	remaining: 5m 20s
1101: learn: 0.03056581	test: 0.0313653	bestTest: 0.0313653 (1101)	total: 3m 6s	remaining: 5m 20s
1102: learn: 0.03056391	test: 0.03136751	bestTest: 0.0313653 (1101)	total: 3m 6s	remaining: 5m 20s
1103: learn: 0.03056286	test: 0.0313663	bestTest: 0.0313653 (1101)	total: 3m 6s	remaining: 5m 19s
1104: learn: 0.03056145	test: 0.03136372	bestTest: 0.03136372 (1104)	total: 3m 6s	remaining: 5m 19s
1105: learn: 0.03055684	test: 0.03136431	bestTest: 0.03136372 (1104)	total: 3m 6s	remaining: 5m 19s
1106: learn

1181: learn: 0.03029024	test: 0.03123175	bestTest: 0.03123135 (1179)	total: 3m 18s	remaining: 5m 5s
1182: learn: 0.03028806	test: 0.03123049	bestTest: 0.03123049 (1182)	total: 3m 18s	remaining: 5m 5s
1183: learn: 0.0302861	test: 0.03123263	bestTest: 0.03123049 (1182)	total: 3m 18s	remaining: 5m 4s
1184: learn: 0.0302858	test: 0.03123283	bestTest: 0.03123049 (1182)	total: 3m 18s	remaining: 5m 4s
1185: learn: 0.03028401	test: 0.03123187	bestTest: 0.03123049 (1182)	total: 3m 18s	remaining: 5m 3s
1186: learn: 0.03028328	test: 0.03123224	bestTest: 0.03123049 (1182)	total: 3m 18s	remaining: 5m 3s
1187: learn: 0.03028078	test: 0.03123078	bestTest: 0.03123049 (1182)	total: 3m 18s	remaining: 5m 3s
1188: learn: 0.03027682	test: 0.03122827	bestTest: 0.03122827 (1188)	total: 3m 18s	remaining: 5m 2s
1189: learn: 0.03027596	test: 0.03122814	bestTest: 0.03122814 (1189)	total: 3m 18s	remaining: 5m 2s
1190: learn: 0.03026681	test: 0.03122237	bestTest: 0.03122237 (1190)	total: 3m 20s	remaining: 5m 4s
11

1264: learn: 0.02999953	test: 0.03112285	bestTest: 0.03112285 (1264)	total: 3m 41s	remaining: 5m 4s
1265: learn: 0.02999182	test: 0.03111938	bestTest: 0.03111938 (1265)	total: 3m 43s	remaining: 5m 5s
1266: learn: 0.02999082	test: 0.03111928	bestTest: 0.03111928 (1266)	total: 3m 43s	remaining: 5m 5s
1267: learn: 0.02999052	test: 0.03112023	bestTest: 0.03111928 (1266)	total: 3m 43s	remaining: 5m 4s
1268: learn: 0.02998668	test: 0.0311187	bestTest: 0.0311187 (1268)	total: 3m 44s	remaining: 5m 6s
1269: learn: 0.02998559	test: 0.0311187	bestTest: 0.0311187 (1268)	total: 3m 44s	remaining: 5m 5s
1270: learn: 0.02997851	test: 0.03111738	bestTest: 0.03111738 (1270)	total: 3m 45s	remaining: 5m 7s
1271: learn: 0.02997809	test: 0.03111734	bestTest: 0.03111734 (1271)	total: 3m 45s	remaining: 5m 6s
1272: learn: 0.02997569	test: 0.03111698	bestTest: 0.03111698 (1272)	total: 3m 45s	remaining: 5m 6s
1273: learn: 0.0299754	test: 0.03111702	bestTest: 0.03111698 (1272)	total: 3m 45s	remaining: 5m 5s
1274:

1350: learn: 0.02978349	test: 0.03109072	bestTest: 0.03108975 (1327)	total: 4m 4s	remaining: 4m 58s
1351: learn: 0.02978254	test: 0.03108966	bestTest: 0.03108966 (1351)	total: 4m 4s	remaining: 4m 57s
1352: learn: 0.02977906	test: 0.03108933	bestTest: 0.03108933 (1352)	total: 4m 4s	remaining: 4m 57s
1353: learn: 0.02977818	test: 0.03108979	bestTest: 0.03108933 (1352)	total: 4m 4s	remaining: 4m 57s
1354: learn: 0.02977414	test: 0.03108987	bestTest: 0.03108933 (1352)	total: 4m 4s	remaining: 4m 57s
1355: learn: 0.02976683	test: 0.0310894	bestTest: 0.03108933 (1352)	total: 4m 6s	remaining: 4m 58s
1356: learn: 0.02976657	test: 0.0310895	bestTest: 0.03108933 (1352)	total: 4m 6s	remaining: 4m 58s
1357: learn: 0.02976593	test: 0.03108976	bestTest: 0.03108933 (1352)	total: 4m 6s	remaining: 4m 57s
1358: learn: 0.02976474	test: 0.03108994	bestTest: 0.03108933 (1352)	total: 4m 6s	remaining: 4m 57s
1359: learn: 0.0297627	test: 0.03108994	bestTest: 0.03108933 (1352)	total: 4m 6s	remaining: 4m 57s
136

In [8]:
def runCatBoost(x_train, y_train,x_test, y_test,test,depth):
    model=CatBoostRegressor(iterations=1000,
                            learning_rate=0.005,
                            depth=depth,
                            loss_function='RMSE',
                            eval_metric='RMSE',
                            random_seed=99,
                            od_type='Iter',
                            od_wait=50)
    model.fit(x_train, y_train, eval_set=(x_test, y_test), use_best_model=True, verbose=False)
    y_pred_train=model.predict(x_test)
    mse=mean_squared_error(y_test, y_pred_train)
    y_pred_test=model.predict(test)
    return y_pred_train,mse,y_pred_test


kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=30)
for dev_index, val_index in kf.split(X):
    dev_X, val_X = X.loc[dev_index], X.loc[val_index]
    dev_y, val_y = Y_bee.loc[dev_index], Y_bee.loc[val_index]
    y_pred_bee,mse_bee,y_pred_test_bee=runCatBoost(dev_X, dev_y, val_X, val_y,test,depth=5)
print("Mean cv score : ", np.mean(mse_bee))








/usr/local/lib/python3.5/dist-packages/catboost/core.py:533: FutureWarning: The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').
  warnings.warn("The 'verbose' parameter is deprecated, use 'logging_level' parameter instead (possible values: 'Silent', 'Verbose', 'Info', 'Debug').", FutureWarning)


0: learn: 1.102872	test: 1.083694	bestTest: 1.083694 (0)	total: 15.8ms	remaining: 19s
1: learn: 1.082074	test: 1.063157	bestTest: 1.063157 (1)	total: 30.6ms	remaining: 18.4s
2: learn: 1.061876	test: 1.043206	bestTest: 1.043206 (2)	total: 45ms	remaining: 17.9s
3: learn: 1.041843	test: 1.023252	bestTest: 1.023252 (3)	total: 65.7ms	remaining: 19.7s
4: learn: 1.022184	test: 1.003659	bestTest: 1.003659 (4)	total: 86.3ms	remaining: 20.6s
5: learn: 1.002949	test: 0.984408	bestTest: 0.984408 (5)	total: 102ms	remaining: 20.3s
6: learn: 0.9837988	test: 0.9655246	bestTest: 0.9655246 (6)	total: 122ms	remaining: 20.8s
7: learn: 0.966075	test: 0.9478471	bestTest: 0.9478471 (7)	total: 137ms	remaining: 20.4s
8: learn: 0.9478323	test: 0.9299541	bestTest: 0.9299541 (8)	total: 153ms	remaining: 20.2s
9: learn: 0.9305361	test: 0.9125935	bestTest: 0.9125935 (9)	total: 176ms	remaining: 21s
10: learn: 0.9132936	test: 0.8954195	bestTest: 0.8954195 (10)	total: 199ms	remaining: 21.5s
11: learn: 0.8960824	test: 0

97: learn: 0.2061061	test: 0.194794	bestTest: 0.194794 (97)	total: 1.81s	remaining: 20.3s
98: learn: 0.2031915	test: 0.1919557	bestTest: 0.1919557 (98)	total: 1.83s	remaining: 20.3s
99: learn: 0.2003461	test: 0.1891396	bestTest: 0.1891396 (99)	total: 1.84s	remaining: 20.3s
100: learn: 0.1975159	test: 0.1864399	bestTest: 0.1864399 (100)	total: 1.86s	remaining: 20.2s
101: learn: 0.1948004	test: 0.1838361	bestTest: 0.1838361 (101)	total: 1.88s	remaining: 20.2s
102: learn: 0.1921938	test: 0.1812318	bestTest: 0.1812318 (102)	total: 1.89s	remaining: 20.1s
103: learn: 0.1896432	test: 0.1787445	bestTest: 0.1787445 (103)	total: 1.91s	remaining: 20.1s
104: learn: 0.1872635	test: 0.1763748	bestTest: 0.1763748 (104)	total: 1.93s	remaining: 20.1s
105: learn: 0.1847672	test: 0.1739651	bestTest: 0.1739651 (105)	total: 1.94s	remaining: 20.1s
106: learn: 0.1823338	test: 0.1716021	bestTest: 0.1716021 (106)	total: 1.96s	remaining: 20s
107: learn: 0.1800646	test: 0.1693025	bestTest: 0.1693025 (107)	total:

191: learn: 0.1001881	test: 0.09402714	bestTest: 0.09402714 (191)	total: 3.42s	remaining: 17.9s
192: learn: 0.09987736	test: 0.09375716	bestTest: 0.09375716 (192)	total: 3.44s	remaining: 17.9s
193: learn: 0.09957019	test: 0.09354086	bestTest: 0.09354086 (193)	total: 3.47s	remaining: 18s
194: learn: 0.09926417	test: 0.09326798	bestTest: 0.09326798 (194)	total: 3.48s	remaining: 18s
195: learn: 0.09900282	test: 0.09308044	bestTest: 0.09308044 (195)	total: 3.5s	remaining: 17.9s
196: learn: 0.09866117	test: 0.09277173	bestTest: 0.09277173 (196)	total: 3.52s	remaining: 17.9s
197: learn: 0.09844016	test: 0.09258453	bestTest: 0.09258453 (197)	total: 3.54s	remaining: 17.9s
198: learn: 0.09820567	test: 0.09240161	bestTest: 0.09240161 (198)	total: 3.56s	remaining: 17.9s
199: learn: 0.09795048	test: 0.09215918	bestTest: 0.09215918 (199)	total: 3.57s	remaining: 17.9s
200: learn: 0.09769218	test: 0.09194018	bestTest: 0.09194018 (200)	total: 3.6s	remaining: 17.9s
201: learn: 0.0975002	test: 0.0917729

277: learn: 0.08921813	test: 0.08502924	bestTest: 0.08502924 (277)	total: 5.01s	remaining: 16.6s
278: learn: 0.08918115	test: 0.0850153	bestTest: 0.0850153 (278)	total: 5.03s	remaining: 16.6s
279: learn: 0.08913922	test: 0.08497344	bestTest: 0.08497344 (279)	total: 5.05s	remaining: 16.6s
280: learn: 0.08911084	test: 0.08495797	bestTest: 0.08495797 (280)	total: 5.07s	remaining: 16.6s
281: learn: 0.08903828	test: 0.08492665	bestTest: 0.08492665 (281)	total: 5.09s	remaining: 16.6s
282: learn: 0.08901113	test: 0.084911	bestTest: 0.084911 (282)	total: 5.11s	remaining: 16.5s
283: learn: 0.08898516	test: 0.08489995	bestTest: 0.08489995 (283)	total: 5.13s	remaining: 16.5s
284: learn: 0.08893887	test: 0.08488285	bestTest: 0.08488285 (284)	total: 5.15s	remaining: 16.5s
285: learn: 0.08890495	test: 0.08486016	bestTest: 0.08486016 (285)	total: 5.17s	remaining: 16.5s
286: learn: 0.08883376	test: 0.0848074	bestTest: 0.0848074 (286)	total: 5.19s	remaining: 16.5s
287: learn: 0.08878326	test: 0.0847664

...

436: learn: 0.07950148	test: 0.09982943	bestTest: 0.09982025 (435)	total: 7.92s	remaining: 13.8s
437: learn: 0.07949264	test: 0.09982199	bestTest: 0.09982025 (435)	total: 7.93s	remaining: 13.8s
438: learn: 0.07948662	test: 0.09982137	bestTest: 0.09982025 (435)	total: 7.95s	remaining: 13.8s
439: learn: 0.07944623	test: 0.09982057	bestTest: 0.09982025 (435)	total: 7.97s	remaining: 13.8s
440: learn: 0.07936876	test: 0.09980682	bestTest: 0.09980682 (440)	total: 7.99s	remaining: 13.8s
441: learn: 0.07936179	test: 0.09980619	bestTest: 0.09980619 (441)	total: 8.01s	remaining: 13.7s
442: learn: 0.07934706	test: 0.09979874	bestTest: 0.09979874 (442)	total: 8.02s	remaining: 13.7s
443: learn: 0.07934309	test: 0.09980252	bestTest: 0.09979874 (442)	total: 8.04s	remaining: 13.7s
444: learn: 0.07934264	test: 0.09980281	bestTest: 0.09979874 (442)	total: 8.05s	remaining: 13.7s
445: learn: 0.07932519	test: 0.09980613	bestTest: 0.09979874 (442)	total: 8.06s	remaining: 13.6s
446: learn: 0.0793161	test: 0.

525: learn: 0.07757588	test: 0.09915679	bestTest: 0.09915679 (525)	total: 9.52s	remaining: 12.2s
526: learn: 0.07756481	test: 0.09914955	bestTest: 0.09914955 (526)	total: 9.53s	remaining: 12.2s
527: learn: 0.07756473	test: 0.09914965	bestTest: 0.09914955 (526)	total: 9.55s	remaining: 12.2s
528: learn: 0.07756467	test: 0.09914974	bestTest: 0.09914955 (526)	total: 9.56s	remaining: 12.1s
529: learn: 0.07756464	test: 0.09914964	bestTest: 0.09914955 (526)	total: 9.58s	remaining: 12.1s
530: learn: 0.07754553	test: 0.09915062	bestTest: 0.09914955 (526)	total: 9.6s	remaining: 12.1s
531: learn: 0.07751707	test: 0.09911856	bestTest: 0.09911856 (531)	total: 9.61s	remaining: 12.1s
532: learn: 0.07748572	test: 0.09911512	bestTest: 0.09911512 (532)	total: 9.63s	remaining: 12s
533: learn: 0.07746124	test: 0.09910953	bestTest: 0.09910953 (533)	total: 9.64s	remaining: 12s
534: learn: 0.07744867	test: 0.09909997	bestTest: 0.09909997 (534)	total: 9.66s	remaining: 12s
535: learn: 0.0774417	test: 0.0990926

617: learn: 0.075937	test: 0.09858178	bestTest: 0.09858178 (617)	total: 11.1s	remaining: 10.5s
618: learn: 0.07593118	test: 0.09857776	bestTest: 0.09857776 (618)	total: 11.1s	remaining: 10.5s
619: learn: 0.07592854	test: 0.09858164	bestTest: 0.09857776 (618)	total: 11.2s	remaining: 10.4s
620: learn: 0.07592827	test: 0.09858198	bestTest: 0.09857776 (618)	total: 11.2s	remaining: 10.4s
621: learn: 0.07592215	test: 0.09858351	bestTest: 0.09857776 (618)	total: 11.2s	remaining: 10.4s
622: learn: 0.07587263	test: 0.0985651	bestTest: 0.0985651 (622)	total: 11.2s	remaining: 10.4s
623: learn: 0.07587229	test: 0.0985632	bestTest: 0.0985632 (623)	total: 11.2s	remaining: 10.4s
624: learn: 0.07585545	test: 0.09854462	bestTest: 0.09854462 (624)	total: 11.2s	remaining: 10.3s
625: learn: 0.07583184	test: 0.09854144	bestTest: 0.09854144 (625)	total: 11.3s	remaining: 10.3s
626: learn: 0.07579141	test: 0.09853154	bestTest: 0.09853154 (626)	total: 11.3s	remaining: 10.3s
627: learn: 0.07577197	test: 0.09852

706: learn: 0.07460824	test: 0.09821122	bestTest: 0.09821122 (706)	total: 12.7s	remaining: 8.88s
707: learn: 0.07460404	test: 0.09820814	bestTest: 0.09820814 (707)	total: 12.8s	remaining: 8.87s
708: learn: 0.07459978	test: 0.09820444	bestTest: 0.09820444 (708)	total: 12.8s	remaining: 8.85s
709: learn: 0.0745884	test: 0.09821178	bestTest: 0.09820444 (708)	total: 12.8s	remaining: 8.83s
710: learn: 0.07455735	test: 0.09820964	bestTest: 0.09820444 (708)	total: 12.8s	remaining: 8.81s
711: learn: 0.0745492	test: 0.0982119	bestTest: 0.09820444 (708)	total: 12.8s	remaining: 8.79s
712: learn: 0.07453711	test: 0.09820738	bestTest: 0.09820444 (708)	total: 12.8s	remaining: 8.77s
713: learn: 0.07453468	test: 0.09821275	bestTest: 0.09820444 (708)	total: 12.9s	remaining: 8.76s
714: learn: 0.07450493	test: 0.09821801	bestTest: 0.09820444 (708)	total: 12.9s	remaining: 8.74s
715: learn: 0.07450492	test: 0.09821802	bestTest: 0.09820444 (708)	total: 12.9s	remaining: 8.72s
716: learn: 0.07448564	test: 0.09

794: learn: 0.07354863	test: 0.09808102	bestTest: 0.09804964 (781)	total: 14.3s	remaining: 7.3s
795: learn: 0.07353455	test: 0.0980872	bestTest: 0.09804964 (781)	total: 14.4s	remaining: 7.29s
796: learn: 0.07353001	test: 0.09808476	bestTest: 0.09804964 (781)	total: 14.4s	remaining: 7.27s
797: learn: 0.07352224	test: 0.09807991	bestTest: 0.09804964 (781)	total: 14.4s	remaining: 7.25s
798: learn: 0.07350955	test: 0.0980729	bestTest: 0.09804964 (781)	total: 14.4s	remaining: 7.23s
799: learn: 0.07349898	test: 0.09807588	bestTest: 0.09804964 (781)	total: 14.4s	remaining: 7.22s
800: learn: 0.07348681	test: 0.09807144	bestTest: 0.09804964 (781)	total: 14.4s	remaining: 7.2s
801: learn: 0.07347682	test: 0.09807283	bestTest: 0.09804964 (781)	total: 14.5s	remaining: 7.18s
802: learn: 0.07347549	test: 0.09807256	bestTest: 0.09804964 (781)	total: 14.5s	remaining: 7.17s
803: learn: 0.07347408	test: 0.09807394	bestTest: 0.09804964 (781)	total: 14.5s	remaining: 7.15s
804: learn: 0.07345595	test: 0.098

In [9]:


sub=pd.DataFrame()

sub["id"]=test_id
sub["formation_energy_ev_natom"]=np.exp(y_pred_test_feen)-1
sub["bandgap_energy_ev"]=np.exp(y_pred_test_bee)-1

import csv
with open('0.00543875322919_.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)

    
np.where(sub<0)
#your_list[368]
#list(sub)[368]


(array([], dtype=int64), array([], dtype=int64))

In [10]:

sub[sub<0]=0.1013936496457506
#mse_total=np.mean([mse_bee,mse_feen])
#print(mse_total)
sub.to_csv(str(22)+"_.csv",index=False)
